In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15
!pip install transformers
!pip install numpy==1.19.5
!pip install 'h5py==2.10.0'
!pip install natsort

In [ ]:
import torch
import transformers as ppb
import tensorflow as tf
import tensorflow_hub as hub
import natsort

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model


from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, MaxPooling1D, Dropout, Conv1D, Input, LSTM, SpatialDropout1D, Bidirectional, Conv1D, MaxPooling2D, Layer

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
import h5py
print(h5py.__version__)

#IF the numpy version not equal to 1.19.5 and h5py version not equal to 2.10.0 restart the runtime and start again from cell 2

In [ ]:
from google.colab import auth
auth.authenticate_user()
!curl https://sdk.cloud.google.com | bash


!gcloud init

In [ ]:
validation = True
gridsearch=False

In [ ]:
!mkdir training_directory
!mkdir validation_directory

In [ ]:
#choose the option depending on what is needed
if validation:

    #!gsutil -m cp gs://masterthesisbert/embeddings_labeled/PsychBERT_con_long/MyBERT_wide_full-training-features_2D_labeled_reddit_500_val* /content/validation_directory/
    #!gsutil -m cp gs://masterthesisbert/embeddings_right/PsychBERT_con_long/MyBERT_wide_full-training-features_2D_labeled_reddit_500_train* /content/training_directory/
    !gsutil -m cp gs://masterthesisbert/embeddings_labeled/BERT/MyBERT_wide_full-training-features_3D_labeled_Aladag* /content/validation_directory/
    !gsutil -m cp gs://masterthesisbert/embeddings_right/BERT/BERT_wide_full-training-features_3D_labeled_Aladag* /content/training_directory/

else:

    #!gsutil -m cp gs://masterthesisbert/embeddings_right/PsychBERT_con_long/MyBERT_wide_full-training-features_3D_labeled_Alada* /content/training_directory/
    #!gsutil -m cp gs://masterthesisbert/embeddings_right/PsychBERT_con_long/MyBERT_wide_full-training-features_2D_labeled_Alada* /content/training_directory/

In [ ]:
import os
if validation:
    arr = os.listdir("/content/training_directory")
    arr = natsort.natsorted(arr)
    print(arr)
    arrval = os.listdir("/content/validation_directory")
    arrval = natsort.natsorted(arrval)
    print(arrval)

else:
    arr = os.listdir("/content/training_directory")
    arr = natsort.natsorted(arr)
    print(arr)

In [ ]:
# load numpy array from csv file
from numpy import loadtxt
import numpy as np

third_dimension = 768

arrays = []
arrays2 = []

In [ ]:
aladag = True
reddit_500 = False
rsdd = False
ten_classes = False

In [ ]:
if aladag:

    length = 10
    length_1 = 1
elif reddit_500:
    length = 5
    length_1 = 2

elif rsdd:
    length = 5
    length_1 = 2

elif ten_classes:
    length = 10
    length_1 = 2

In [ ]:
if(validation==True):
    for i in range(len(arr[0:length])):
        print(arr[i])
        loaded_arr = np.loadtxt('/content/training_directory'+"/"+arr[i])
        arrays.append(loaded_arr)
        print(i)
    
    for i in range(length_1):
      print(arrval[i])
      loaded_arr2 = np.loadtxt('/content/validation_directory'+"/"+arrval[i])
      arrays2.append(loaded_arr2)
      print(i)
    
else:
    
    #choose the right one for validation
    for i in range(len(arr[0:length])):
       
        loaded_arr = np.loadtxt('/content/training_directory'+"/"+arr[i])
        arrays.append(loaded_arr)
        print(i)

In [ ]:
threeD = True
if threeD:
    
    concatenated_array = np.array([]).reshape(0, arrays[2].shape[1])

    for array in arrays:

        concatenated_array = np.concatenate([concatenated_array, array], axis=0)

    del arrays

    load_original_arr = concatenated_array.reshape(
        concatenated_array.shape[0], concatenated_array.shape[1] // third_dimension, third_dimension)
    del concatenated_array
    data_train_x = load_original_arr
    del load_original_arr


    
    concatenated_array2 = np.array([]).reshape(0, arrays2[0].shape[1])

    for array in arrays2:

        concatenated_array2 = np.concatenate([concatenated_array2, array], axis=0)

    del arrays2

    load_original_arr2 = concatenated_array2.reshape(
        concatenated_array2.shape[0], concatenated_array2.shape[1] // third_dimension, third_dimension)
    del concatenated_array2
    data_train_x2 = load_original_arr2

else:

    if(validation==False):

        concatenated_array = np.array([]).reshape(0, 768)
        # Create an array to return to

        for array in arrays:

            concatenated_array = np.concatenate([concatenated_array, array], axis=0)
        data_train_x = concatenated_array
    else: 
        concatenated_array = np.array([]).reshape(0, 768)
        # Create an array to return to

        for array in arrays:

            concatenated_array = np.concatenate([concatenated_array, array], axis=0)
        data_train_x = concatenated_array

        concatenated_array2 = np.array([]).reshape(0, 768)
        # Create an array to return to

        for array in arrays2:

            concatenated_array2 = np.concatenate([concatenated_array2, array], axis=0)
        data_train_x2 = concatenated_array2


In [ ]:
import pandas as pd
import tensorflow as tf

if aladag:  

    !gsutil cp gs://masterthesisbert/Aladag_sample_preprocessed.csv  /content/Aladag_sample_preprocessed.csv
    
    df_name = "/content/Aladag_sample_preprocessed.csv"
    df = pd.read_csv(df_name)

    !gsutil cp gs://masterthesisbert/Aladag_labeled_preprocessed.csv  /content/Aladag_labeled_preprocessed.csv

    df_name2 = "/content/Aladag_labeled_preprocessed.csv"
    df2 = pd.read_csv(df_name2)
    df2

    train_y_tensor = torch.tensor(df[0:10000].binary_annotation.values)
    train_y_tensor2 = torch.tensor(df2[0:10000].binary_annotation.values)


elif reddit_500:
    !gsutil cp gs://masterthesisbert/reddit_500_final_val.csv  /content/reddit_500_final_val.csv

    df_name2 = "/content/reddit_500_final_val.csv"
    df2 = pd.read_csv(df_name2)
    df2

    !gsutil cp gs://masterthesisbert/reddit_500_final_train.csv  /content/reddit_500_final_train.csv

    df_name = "/content/reddit_500_final_train.csv"
    df = pd.read_csv(df_name)
    df

    train_y_tensor = torch.tensor(df[0:10000].Label.values)
    train_y_tensor2 = torch.tensor(df2[0:10000].Label.values)

elif rsdd:

    !gsutil cp gs://masterthesisbert/rsdd_valid_preprocessed.csv  /content/rsdd_valid_preprocessed.csv

    df_name2 = "/content/rsdd_valid_preprocessed.csv"
    df2 = pd.read_csv(df_name2)
    df2

    !gsutil cp gs://masterthesisbert/rsdd_train_preprocessed.csv  /content/rsdd_train_preprocessed.csv

    df_name = "/content/rsdd_train_preprocessed.csv"
    df = pd.read_csv(df_name)

    train_y_tensor = torch.tensor(df[0:5000].label.values)
    train_y_tensor2 = torch.tensor(df2[0:2000].label.values)


elif ten_classes:

    
    !gsutil cp gs://masterthesisbert/df_mental_valid_preprocessed_preprocessed.csv  /content/df_mental_valid_preprocessed_preprocessed.csv

    df_name2 = "/content/df_mental_valid_preprocessed_preprocessed.csv"
    df2 = pd.read_csv(df_name2)
    df2

    !gsutil cp gs://masterthesisbert/df_mental_balanced_preprocessed_preprocessed.csv  /content/df_mental_balanced_preprocessed_preprocessed.csv 
    df_name = "/content/df_mental_balanced_preprocessed_preprocessed.csv"
    df = pd.read_csv(df_name)
    df
    
    df.cc = pd.Categorical(df.label)
    df["cat"] = df.cc.codes
    df2.cc = pd.Categorical(df2.label)
    df2["cat"] = df2.cc.codes  
    train_y_tensor = torch.tensor(df.cat.values)     
        
    train_y_tensor2 = torch.tensor(df2.cat.values)

In [ ]:
import numpy as np

from tensorflow import keras
from tensorflow.python.keras.layers import Input, Dense,RepeatVector, TimeDistributed, Dense, Dropout, LSTM, Bidirectional
from tensorflow.python.keras.models import Sequential

import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
print(np.__version__)

In [ ]:
# Conctruct autoencoder for training test to find anomalous days in the data
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
import os
import random as rn 
from tensorflow.compat.v1.keras import backend as K
number = 2
os.environ['PYTHONHASHSEED']=str(number)
#Set random seed for numpy, python and tensorflow
np.random.seed(number)
rn.seed(number)
tf.set_random_seed(number)
# Set the number of threads to 1 
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
# Its an example of an autoencoder. I might use it later. 

In [ ]:
def unison_shuffled_copies(a, b, just_indices=False):
        assert len(a) == len(b)
        p = numpy.random.permutation(len(a))
        return a[p], b[p], p 

In [ ]:
permutation_done = True


if permutation_done and reddit_500:
    !gsutil cp gs://masterthesisbert/parameter_optimization/permutation.csv  /content/permutation.csv
    from numpy import loadtxt
    p = loadtxt('permutation.csv', delimiter=',')
    data_train_x_permut, train_y_tensor_permut = data_train_x[p.astype(int)], train_y_tensor[p.astype(int)]


elif(permutation_done==False and reddit_500==True):
    
    data_train_x_permut, train_y_tensor_permut, p = unison_shuffled_copies(data_train_x, train_y_tensor)

    from numpy import savetxt
    savetxt('permutation.csv', p, delimiter=',')

    from numpy import loadtxt
    p = loadtxt('permutation.csv', delimiter=',')
    p
    #!gsutil cp /content/permutation.csv gs://masterthesisbert/parameter_optimization/permutation.csv

In [ ]:
from tensorflow.keras.utils import to_categorical

if ten_classes:
    train_y_tensor = to_categorical(train_y_tensor)
    train_y_tensor2 = to_categorical(train_y_tensor2)


#Novelty Detection

In [ ]:
novelty_detection = True
if novelty_detection:
    #for the novelty detection the data is already permutated
    !gsutil cp gs://masterthesisbert/anomaly_indices/BERT_Aladag_noisy1.txt /content/
    #!gsutil cp gs://masterthesisbert/anomaly_indices/PsychBERT1_reddit_500_noisy1.txt /content/

    my_file = open("/content/BERT_Aladag_noisy1.txt", "r")
    content = my_file.read()
    normal_data = content.split("\n")
    normal_data.pop()

    normal_data = [int(i) for i in normal_data]
    normal_data = sorted(normal_data)

    if reddit_500:
        data_train_x_permut = data_train_x_permut[normal_data]
        train_y_tensor_permut = train_y_tensor_permut[normal_data]
    elif aladag:
        data_train_x = data_train_x[normal_data]
        train_y_tensor = train_y_tensor[normal_data]


In [ ]:
def summarize_results(grid_result, model_name, result_to_file = True):
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    if result_to_file == True:
        string_to_save = f"Best: {grid_result.best_score_} using {grid_result.best_params_} \n" 
        for mean, stdev, param in zip(means, stds, params):
            string_to_save = string_to_save + f"{mean} ({stdev}) with: {param} \n" 
        text_results = open(model_name + ".txt", "w")
        n = text_results.write(string_to_save)
        text_results.close()


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#Simple Dense 2D

In [ ]:
# Fully Dense Network

def test_dense(data_train_x=data_train_x, number_of_classes = 2, learn_rate=0.001, momentum = 0.2, dropout_rate=0.3):
    number = 2
    os.environ['PYTHONHASHSEED']=str(number)
    #Set random seed for numpy, python and tensorflow
    np.random.seed(number)
    rn.seed(number)
    tf.set_random_seed(number)
    # Set the number of threads to 1 
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

    dense = Sequential()
    dense.add(Input(shape=(768,)))
    dense.add(keras.layers.Dropout(rate=dropout_rate))

    dense.add(Dense(384, activation='relu',  kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
    dense.add(keras.layers.Dropout(rate=dropout_rate))

    dense.add(Dense(128, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
    dense.add(keras.layers.Dropout(rate=dropout_rate))

    dense.add(Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
    dense.add(keras.layers.Dropout(rate=dropout_rate))
    if number_of_classes==2:

        dense.add(Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
        lossi = 'binary_crossentropy'

    elif number_of_classes==10:
        dense.add(Dense(10, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

        lossi = "categorical_crossentropy"

        
    #opt = keras.optimizers.Adam(momentum=momentum)
    optimizer = keras.optimizers.Adam(learning_rate=learn_rate)

    dense.compile(optimizer=optimizer,
                  loss=lossi,
                  metrics=['accuracy'])

    #mc = ModelCheckpoint(dense_path + ".h5", monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    dense.summary()

    return dense

In [ ]:
validation = False

In [ ]:
if gridsearch:
    if (validation==False):
        if reddit_500:
        
            model_autoencoder_LSTM = KerasClassifier(build_fn=test_dense, epochs=5, learn_rate=0.0001, dropout_rate=0.3)

        if ten_classes:
            model_autoencoder_LSTM = KerasClassifier(build_fn=test_dense, epochs=5, learn_rate=0.0001, dropout_rate=0.3,  number_of_classes = 10)



        learn_rate = [0.0001, 0.00001]
        dropout_rate = [0.4, 0.5, 0.55]
        epochs= [100, 120]
        params=dict(learn_rate = learn_rate,
                    dropout_rate = dropout_rate,
                    epochs=epochs

        )
        grid = GridSearchCV(estimator=model_autoencoder_LSTM, param_grid=params, cv=5)

    if (validation==False):
        #if dataset is reddit 500 use data_train_x_permut and train_y_tensor_permut otherwise use data_train_x
        if reddit_500:

            grid_result = grid.fit(data_train_x_permut, train_y_tensor_permut)

        else:
            grid_result = grid.fit(data_train_x, train_y_tensor)

        model_name = "PsychBERT_con_long_ten_classes_Dense"
        model_name_txt = model_name + ".txt"
        summarize_results(grid_result, model_name)

        !gsutil cp /content/{model_name_txt} gs://masterthesisbert/parameter_optimization/{model_name_txt}

In [ ]:
validation = True

In [ ]:
if validation:
    final_dense = test_dense(data_train_x_permut, learn_rate=0.0001, dropout_rate=0.4, number_of_classes = 2)
    final_dense.fit(data_train_x_permut, train_y_tensor_permut,  epochs = 120)

In [ ]:
def accuracy(y_true, y_pred):
    
    
    """Function to calculate accuracy
    -> param y_true: list of true values
    -> param y_pred: list of predicted values
    -> return: accuracy score
    """
       
    # Intitializing variable to store count of correctly predicted classes
    correct_predictions = 0
    
    for yt, yp in zip(y_true, y_pred):
        
        if np.array_equal(yt, yp):
            
            correct_predictions += 1
    
    #returns accuracy
    return correct_predictions / len(y_true)

In [ ]:
if validation:
    predicted2 = final_dense.predict(data_train_x2)
    if ten_classes: 
        b = np.zeros_like(predicted2)
        b[np.arange(len(predicted2)), predicted2.argmax(1)] = 1
        predicted2 = b
    else:
        predicted2 = np.where(predicted2<0.5, 0, 1)


    model_text = ""

    model_text = model_text + "PsychBERT2 Dense trained on 10 classes validated on 10 classes"
    print(model_text)
    #accuracy

    model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor2, predicted2), 5)) 
    
    print("Accuracy:", round(accuracy_score(train_y_tensor2, predicted2), 5))

    if ten_classes:
        #precision
        model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("Precision:", round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5))
        #recall
        model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("Recall:", round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5))

        #F1Score
        model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("F1score:", round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5))


    else:

        #precision
        model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2), 5)) 

        print("Precision:", round(precision_score(train_y_tensor2, predicted2), 5))
        #recall
        model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2), 5)) 

        print("Recall:", round(recall_score(train_y_tensor2, predicted2), 5))

        #F1Score
        model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2), 5)) 

        print("F1score:", round(f1_score(train_y_tensor2, predicted2), 5)) 

#LSTM-CNN

In [ ]:
def create_lstm_cnn(data_train_x=data_train_x, training = True, number_of_classes=2, learn_rate=0.001, momentum = 0.2, dropout_rate=0.3):
    number = 2
    os.environ['PYTHONHASHSEED']=str(number)
    #Set random seed for numpy, python and tensorflow
    np.random.seed(number)
    rn.seed(number)
    tf.set_random_seed(number)
    # Set the number of threads to 1 
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)


    lstm_cnn = Sequential() 

    lstm_cnn.add(Input(shape=(256,768)))

    lstm_cnn.add(Dropout(0.5)) 
    lstm_cnn.add(LSTM(units=100, return_sequences=True)) 
    lstm_cnn.add(Conv1D(3, (8,), padding='same', activation='relu')) 
    lstm_cnn.add(MaxPooling1D(2))
    lstm_cnn.add(Flatten())

    if number_of_classes==2:

        lstm_cnn.add(Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
        lossi = 'binary_crossentropy'

    elif number_of_classes==10 and training == True:
        lstm_cnn.add(Dense(10, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

        lossi = "categorical_crossentropy"

    elif number_of_classes==10 and training == False:
      
        lossi = "categorical_crossentropy"
        lstm_cnn.add(Dense(10, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))


    optimizer = keras.optimizers.Adam(learning_rate=learn_rate)

    lstm_cnn.compile(optimizer=optimizer,
                  loss=lossi,
                  metrics=['accuracy'])

    lstm_cnn.summary()

    return lstm_cnn

In [ ]:
if gridsearch:

    if ten_classes:
        lstm_cnn = KerasClassifier(build_fn=create_lstm_cnn, epochs=2, learn_rate=0.0001, number_of_classes=10 )

    else:

        lstm_cnn = KerasClassifier(build_fn=create_lstm_cnn, epochs=2, learn_rate=0.0001)

    learn_rate = [0.0001]

    epochs= [15, 20]
    params=dict(learn_rate = learn_rate,
                epochs=epochs

    )
    grid = GridSearchCV(estimator=lstm_cnn, param_grid=params, cv=5)

    if reddit_500:
        
        grid_result = grid.fit(data_train_x_permut, train_y_tensor_permut)

    else:
        grid_result = grid.fit(data_train_x, train_y_tensor)


    model_name = "PsychBERT1_10classes_LSTM-CNN_part2"
    model_name_txt = model_name + ".txt"
    summarize_results(grid_result, model_name)

    !gsutil cp /content/{model_name_txt} gs://masterthesisbert/parameter_optimization/{model_name_txt}

In [ ]:
#depeonding on the expected functionality set the validation true or false 
validation=False

In [ ]:
#if train-test split, then this true (if gridsearch = False )
train_different_manner = True

In [ ]:
# each time the training set is the 4/5 of the normal data or chosen size
train_test_split = int(round(len(data_train_x)/5, 0)*4)

# for example 
#train_test_split = 4000
train_test_split

7560

In [ ]:
if train_different_manner:

    learn_rate = [0.0001, 0.00001]

    epochs= [5, 10]

    for i in range(len(learn_rate)):

        for j inn range(len(epochs)):
          
            final_lstm_cnn = create_lstm_cnn(data_train_x[:train_test_split], learn_rate=learn_rate[i], number_of_classes=2)
            final_lstm_cnn.fit(data_train_x[:train_test_split], train_y_tensor[:train_test_split],  epochs = epochs[j])

            predicted2 = final_lstm_cnn.predict(data_train_x[train_test_split:])
            print(learn_rate[i], epochs[j])

            if ten_classes: 
                b = np.zeros_like(predicted2)
                b[np.arange(len(predicted2)), predicted2.argmax(1)] = 1
                predicted2 = b
            else:
                predicted2 = np.where(predicted2<0.5, 0, 1)

            if(i==0 and j==0):

                model_text = "" + "\n"
            else:
                model_text = model_text + "\n"

            #accuracy
            model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5)) 
            print("Accuracy:", round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5))
            if ten_classes:
                  #precision
                  model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("Precision:", round(precision_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))
                  #recall
                  model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("Recall:", round(recall_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))

                  #F1Score
                  model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("F1score:", round(f1_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))


            else:

                  #precision
                  model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("Precision:", round(precision_score(train_y_tensor[train_test_split:], predicted2), 5))
                  #recall
                  model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("Recall:", round(recall_score(train_y_tensor[train_test_split:], predicted2), 5))

                  #F1Score
                  model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("F1score:", round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 


    print(model_text)

    name = "Noise_Results_PsychBERT1_LSTM-CNN_Aladag_notcross.txt"

    with open(name, "w") as text_file:
            text_file.write(model_text)

    !gsutil cp /content/{name} gs://masterthesisbert/parameter_optimization

In [ ]:
final_lstm_cnn = create_lstm_cnn(data_train_x, learn_rate=0.0001, number_of_classes=2)
final_lstm_cnn.fit(data_train_x, train_y_tensor,  epochs = 15)

In [ ]:
predicted2 = final_lstm_cnn.predict(data_train_x2)

if ten_classes: 
        b = np.zeros_like(predicted2)
        b[np.arange(len(predicted2)), predicted2.argmax(1)] = 1
        predicted2 = b
else:
    predicted2 = np.where(predicted2<0.5, 0, 1)

model_text = "" + "\n"

model_text = model_text + "PsychBERT2 LSTM-CNN trained on rsdd"
print(model_text)
#accuracy
model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor2, predicted2), 5)) 
print("Accuracy:", round(accuracy_score(train_y_tensor2, predicted2), 5))
if ten_classes:
      #precision
      model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

      print("Precision:", round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5))
      #recall
      model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

      print("Recall:", round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5))

      #F1Score
      model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

      print("F1score:", round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5))

else:

      #precision
      model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2), 5)) 

      print("Precision:", round(precision_score(train_y_tensor2, predicted2), 5))
      #recall
      model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2), 5)) 

      print("Recall:", round(recall_score(train_y_tensor2, predicted2), 5))

      #F1Score
      model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2), 5)) 

      print("F1score:", round(f1_score(train_y_tensor2, predicted2), 5)) 

# LSTM-Attention-CNN

In [ ]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal") 
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="zeros")
        super(Attention, self).build(input_shape)


    def call(self,x):

        e = K.tanh(K.dot(x, self.W) +self.b) 
        a = K.softmax(e, axis=1) 
        output = x*a

        if self.return_sequences:
            return output

        return K.sum(output, axis=1)

In [ ]:
def create_lstm_atten_cnn(data_train_x=data_train_x, number_of_classes=2, training = True, learn_rate=0.001, momentum = 0.2, dropout_rate=0.3):
    number = 2
    os.environ['PYTHONHASHSEED']=str(number)
    #Set random seed for numpy, python and tensorflow
    np.random.seed(number)
    rn.seed(number)
    tf.set_random_seed(number)
    # Set the number of threads to 1 
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)


    lstm_atten_cnn = Sequential() 

    lstm_atten_cnn.add(Input(shape=(256,768)))

    lstm_atten_cnn.add(Dropout(0.5)) 
    lstm_atten_cnn.add(LSTM(units=100, return_sequences=True)) 
    lstm_atten_cnn.add(Attention(return_sequences=True)) 
    lstm_atten_cnn.add(Conv1D(3, (8,), padding='same', activation='relu')) 
    lstm_atten_cnn.add(MaxPooling1D(2))
    lstm_atten_cnn.add(Flatten()) 

    if number_of_classes==2:

        lstm_atten_cnn.add(Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
        lossi = 'binary_crossentropy'

    elif number_of_classes==10 and training == True:
        lstm_atten_cnn.add(Dense(10, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

        lossi = "categorical_crossentropy"

    elif number_of_classes==10 and training == False:
      
        lossi = "categorical_crossentropy"
        lstm_atten_cnn.add(Dense(10, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

    optimizer = keras.optimizers.Adam(learning_rate=learn_rate)

    lstm_atten_cnn.compile(optimizer=optimizer,
                  loss=lossi,
                  metrics=['accuracy'])

    #mc = ModelCheckpoint(dense_path + ".h5", monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    lstm_atten_cnn.summary()


    return lstm_atten_cnn

In [ ]:
validation=True

In [ ]:


if gridsearch:
    if (validation==False):

        if ten_classes:

            lstm_atten_cnn = KerasClassifier(build_fn=create_lstm_atten_cnn, epochs=2, learn_rate=0.0001, number_of_classes=10)

        else: 
            lstm_atten_cnn = KerasClassifier(build_fn=create_lstm_atten_cnn, epochs=2, learn_rate=0.0001)

        learn_rate = [0.00001]

        epochs= [20]
        params=dict(learn_rate = learn_rate,
                  epochs=epochs

        )
        grid = GridSearchCV(estimator=lstm_atten_cnn, param_grid=params, cv=5)

        if reddit_500:
    
            grid_result = grid.fit(data_train_x_permut, train_y_tensor_permut)

        else:
            grid_result = grid.fit(data_train_x, train_y_tensor)

        model_name = "PsychBERT2_10classes_LSTM-attention-CNN_part2"
        model_name_txt = model_name + ".txt"
        summarize_results(grid_result, model_name)

        !gsutil cp /content/{model_name_txt} gs://masterthesisbert/parameter_optimization/{model_name_txt} 

In [ ]:
validation=True

In [ ]:
train_different_manner = True

In [ ]:
if train_different_manner:

    learn_rate = [0.0001, 0.00001]

    epochs= [5, 10]

    for i in range(len(learn_rate)):

        for j inn range(len(epochs)):
          
            final_lstm_a_cnn = lstm_atten_cnn(data_train_x[:train_test_split], learn_rate=learn_rate[i], number_of_classes=2)
            final_lstm_a_cnn.fit(data_train_x[:train_test_split], train_y_tensor[:train_test_split],  epochs = epochs[j])

            predicted2 = final_lstm_a_cnn.predict(data_train_x[train_test_split:])
            print(learn_rate[i], epochs[j])

            if ten_classes: 
                b = np.zeros_like(predicted2)
                b[np.arange(len(predicted2)), predicted2.argmax(1)] = 1
                predicted2 = b
            else:
                predicted2 = np.where(predicted2<0.5, 0, 1)

            if(i==0 and j==0):

                model_text = "" + "\n"
            else:
                model_text = model_text + "\n"

            #accuracy
            model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5)) 
            print("Accuracy:", round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5))
            if ten_classes:
                  #precision
                  model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("Precision:", round(precision_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))
                  #recall
                  model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("Recall:", round(recall_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))

                  #F1Score
                  model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("F1score:", round(f1_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))


            else:

                  #precision
                  model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("Precision:", round(precision_score(train_y_tensor[train_test_split:], predicted2), 5))
                  #recall
                  model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("Recall:", round(recall_score(train_y_tensor[train_test_split:], predicted2), 5))

                  #F1Score
                  model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("F1score:", round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 


    print(model_text)

    name = "Noise_Results_PsychBERT1_LSTM-Attention-CNN_Aladag_notcross.txt"

    with open(name, "w") as text_file:
            text_file.write(model_text)

    !gsutil cp /content/{name} gs://masterthesisbert/parameter_optimization

In [ ]:
#if (validation==True):
final_lstm_attention_cnn = create_lstm_atten_cnn(data_train_x, learn_rate=0.0001, number_of_classes=2)
final_lstm_attention_cnn.fit(data_train_x, train_y_tensor,  epochs = 25)

In [ ]:
validation=True

In [ ]:
if validation:
    predicted2 = final_lstm_attention_cnn.predict(data_train_x2)

    if ten_classes: 
        b = np.zeros_like(predicted2)
        b[np.arange(len(predicted2)), predicted2.argmax(1)] = 1
        predicted2 = b
    else:
        predicted2 = np.where(predicted2<0.5, 0, 1)

    model_text = "model_text" + "\n"

    model_text = model_text + "PsychBERT2 LSTM-Atten-CNN trained on rsdd "
    print(model_text)
    #accuracy
    model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor2, predicted2), 5)) 
    print("Accuracy:", round(accuracy_score(train_y_tensor2, predicted2), 5))
    if ten_classes:
        #precision
        model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("Precision:", round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5))
        #recall
        model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("Recall:", round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5))

        #F1Score
        model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("F1score:", round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5))


    else:

        #precision
        model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2), 5)) 

        print("Precision:", round(precision_score(train_y_tensor2, predicted2), 5))
        #recall
        model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2), 5)) 

        print("Recall:", round(recall_score(train_y_tensor2, predicted2), 5))

        #F1Score
        model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2), 5)) 

        print("F1score:", round(f1_score(train_y_tensor2, predicted2), 5)) 


#Bi-LSTM

In [ ]:
def create_bilstm(data_train_x=data_train_x, number_of_classes=2, learn_rate=0.0001, training = True, momentum = 0.2, dropout_rate=0.3):
    # Bi-LSTM
    bilstm = Sequential()

    bilstm_path = "bilstm"

    pool_size = 2

    bilstm.add(Input(shape=(256,768)))
    bilstm.add(Bidirectional(LSTM(20, return_sequences=True, dropout=dropout_rate, recurrent_dropout=0.2)))
    bilstm.add(MaxPooling1D(pool_size = pool_size))
    bilstm.add(Flatten())
    bilstm.add(Dropout(0.5)) 

    if number_of_classes==2:

        bilstm.add(Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))
        lossi = 'binary_crossentropy'

    elif number_of_classes==10 and training == True:
 
        bilstm.add(Dense(10, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

        lossi = "categorical_crossentropy"

    elif number_of_classes==10 and training == False:
      
        lossi = "categorical_crossentropy"
        bilstm.add(Dense(10, activation='sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))


    optimizer = keras.optimizers.Adam(learning_rate=learn_rate)

    bilstm.compile(optimizer=optimizer,
                  loss=lossi,
                  metrics=['accuracy'])


    bilstm.summary()
    return bilstm

In [ ]:
if gridsearch:
    if (validation==False):
        if ten_classes:

            bilstm = KerasClassifier(build_fn=create_bilstm, epochs=2, learn_rate=0.0001, number_of_classes=10)

        else: 
            bilstm = KerasClassifier(build_fn=create_bilstm, epochs=2, learn_rate=0.0001)


        learn_rate = [0.00001]

        epochs= [20]
        params=dict(learn_rate = learn_rate,
                    epochs=epochs)
        
        grid = GridSearchCV(estimator=bilstm, param_grid=params, cv=5)

        if reddit_500:
    
            grid_result = grid.fit(data_train_x_permut, train_y_tensor_permut)

        else:
            grid_result = grid.fit(data_train_x, train_y_tensor)

        model_name = "PsychBERT2_10classes_BiLSTM_part2"
        model_name_txt = model_name + ".txt"
        summarize_results(grid_result, model_name)

In [ ]:
if (validation==False):
    !gsutil cp /content/{model_name_txt} gs://masterthesisbert/parameter_optimization/{model_name_txt}  


In [ ]:
validation = True  

In [ ]:
train_different_manner = True

In [ ]:
# each time the training set is the 4/5 of the normal data
train_test_split = int(round(len(data_train_x)/5, 0)*4)
train_test_split

In [ ]:
if reddit_500:
    data_train_x = data_train_x_permut
    train_y_tensor = train_y_tensor_permut

In [ ]:
if train_different_manner:

    learn_rate = [0.0001, 0.00001]

    epochs= [5, 10]

    for i in range(len(learn_rate)):

        for j in range(len(epochs)):
          
            final_bi_lstm = create_bilstm(data_train_x[:train_test_split], learn_rate=learn_rate[i], number_of_classes=2)
            final_bi_lstm.fit(data_train_x[:train_test_split], train_y_tensor[:train_test_split],  epochs = epochs[j])

            predicted2 = final_bi_lstm.predict(data_train_x[train_test_split:])
            print(learn_rate[i], epochs[j])

            if ten_classes: 
                b = np.zeros_like(predicted2)
                b[np.arange(len(predicted2)), predicted2.argmax(1)] = 1
                predicted2 = b
            else:
                predicted2 = np.where(predicted2<0.5, 0, 1)

            if(i==0 and j==0):

                model_text = "" + "\n"
            else:
                model_text = model_text + "\n"

            #accuracy
            model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5)) 
            print("Accuracy:", round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5))
            if ten_classes:
                  #precision
                  model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("Precision:", round(precision_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))
                  #recall
                  model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("Recall:", round(recall_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))

                  #F1Score
                  model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5)) 

                  print("F1score:", round(f1_score(train_y_tensor[train_test_split:], predicted2, average='weighted'), 5))


            else:

                  #precision
                  model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("Precision:", round(precision_score(train_y_tensor[train_test_split:], predicted2), 5))
                  #recall
                  model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("Recall:", round(recall_score(train_y_tensor[train_test_split:], predicted2), 5))

                  #F1Score
                  model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 

                  print("F1score:", round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 


    print(model_text)

    name = "Noise_Results_BERT_BiLSTM_Aladag_notcross.txt"

    with open(name, "w") as text_file:
            text_file.write(model_text)

    !gsutil cp /content/{name} gs://masterthesisbert/parameter_optimization

In [ ]:
if validation:
    final_bi_lstm = create_bilstm(data_train_x, learn_rate=0.0001, number_of_classes=2)
    final_bi_lstm.fit(data_train_x, train_y_tensor,  epochs = 10)

In [ ]:
if validation:   
    predicted2 = final_bi_lstm.predict(data_train_x2)

    if ten_classes: 
        b = np.zeros_like(predicted2)
        b[np.arange(len(predicted2)), predicted2.argmax(1)] = 1
        predicted2 = b
    else:
        predicted2 = np.where(predicted2<0.5, 0, 1)

    model_text = "model_text" + "\n"

    model_text = model_text + "BERT Bi-LSTM trained rsdd"
    print(model_text)
    #accuracy
    model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor2, predicted2), 5)) 
    print("Accuracy:", round(accuracy_score(train_y_tensor2, predicted2), 5))
    if ten_classes:
        #precision
        model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("Precision:", round(precision_score(train_y_tensor2, predicted2, average='weighted'), 5))
        #recall
        model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("Recall:", round(recall_score(train_y_tensor2, predicted2, average='weighted'), 5))

        #F1Score
        model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5)) 

        print("F1score:", round(f1_score(train_y_tensor2, predicted2, average='weighted'), 5))


    else:

        #precision
        model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2), 5)) 

        print("Precision:", round(precision_score(train_y_tensor2, predicted2), 5))
        #recall
        model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2), 5)) 

        print("Recall:", round(recall_score(train_y_tensor2, predicted2), 5))

        #F1Score
        model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2), 5)) 

        print("F1score:", round(f1_score(train_y_tensor2, predicted2), 5)) 